In [1]:
from pprint import pprint
import onnx
from onnx import helper
from onnx import TensorProto
import numpy as np

In [2]:
in_dim = 8
out_dim = 8
def make_layer(in_dim, out_dim, idx):
    # Create one input (ValueInfoPro    to)
    X = helper.make_tensor_value_info(f"X_{idx}", TensorProto.INT32, [in_dim])
    w = helper.make_tensor(f"weight_{idx}", TensorProto.INT32, [out_dim, in_dim], np.random.randint(-10, 10, (out_dim, in_dim)).astype(np.int32).tobytes(), raw=True)
    b = helper.make_tensor(f"bias_{idx}", TensorProto.INT32, [out_dim], np.random.randint(-10, 10, out_dim).astype(np.int32).tobytes(),  raw=True)

    # Create one output (ValueInfoProto)
    Y = helper.make_tensor_value_info(f"Y_{idx}", TensorProto.INT32, [8])

    mmnode = helper.make_node(
        "MatMul",
        [f"X_{idx}", f"weight_{idx}"],
        [f"XMM_{idx}"],
        name=f"mm_{idx}"
    )

    biasnode = helper.make_node(
        "Add",
        [f"XMM_{idx}", f"bias_{idx}"],
        [f"XMMB_{idx}"],
        name=f"add_{idx}"
    )

    relunode = helper.make_node(
        "Relu",
        [f"XMMB_{idx}"],
        [f"Y_{idx}"],
        name=f"relu_{idx}"
    )

    # Create the graph (GraphProto)
    graph_def = helper.make_graph(
        [mmnode, biasnode, relunode],
        f"testmodel_{idx}",
        [X],
        [Y],
        [w, b]
    )

    # Create the model (ModelProto)
    opset = onnx.OperatorSetIdProto()
    opset.version = 14
    return helper.make_model(graph_def, opset_imports = [opset])

model_def = make_layer(8, 8, 0)
print(f"The model is:\n{model_def}")
onnx.checker.check_model(model_def)
print("The model is checked!")

The model is:
ir_version: 9
graph {
  node {
    input: "X_0"
    input: "weight_0"
    output: "XMM_0"
    name: "mm_0"
    op_type: "MatMul"
  }
  node {
    input: "XMM_0"
    input: "bias_0"
    output: "XMMB_0"
    name: "add_0"
    op_type: "Add"
  }
  node {
    input: "XMMB_0"
    output: "Y_0"
    name: "relu_0"
    op_type: "Relu"
  }
  name: "testmodel_0"
  initializer {
    dims: 8
    dims: 8
    data_type: 6
    name: "weight_0"
    raw_data: "\374\377\377\377\376\377\377\377\367\377\377\377\004\000\000\000\t\000\000\000\003\000\000\000\004\000\000\000\004\000\000\000\000\000\000\000\002\000\000\000\373\377\377\377\000\000\000\000\005\000\000\000\002\000\000\000\374\377\377\377\367\377\377\377\377\377\377\377\004\000\000\000\367\377\377\377\006\000\000\000\t\000\000\000\366\377\377\377\367\377\377\377\001\000\000\000\005\000\000\000\001\000\000\000\005\000\000\000\007\000\000\000\377\377\377\377\004\000\000\000\004\000\000\000\367\377\377\377\376\377\377\377\366\377\377\3

In [3]:
def make_multilayer(n_layers, layer_widths = None):
    if layer_widths == None:
        layer_widths = [n_layers for _ in range(2*n_layers)]
    layers = [make_layer(layer_widths[2*i], layer_widths[2*i+1], i) for i in range(n_layers)]
    n_params = sum(layer_widths[2*i]*layer_widths[2*i + 1] + layer_widths[2*i+1] for i in range(n_layers//2))
    model = layers[0]
    for idx, l in enumerate(layers[1:-1]):
        model = onnx.compose.merge_models(model, l, [(f"Y_{idx}", f"X_{idx+1}")])
    if n_layers > 1:
        model = onnx.compose.merge_models(model, layers[-1], [(f"Y_{n_layers-2}", f"X_{n_layers-1}")])
    return model, n_params
print([make_multilayer(i)[1] for i in range(2, 11)])
onnx.checker.check_model(make_multilayer(3)[0])

[6, 12, 40, 60, 126, 168, 288, 360, 550]


In [6]:
from compiler import parsemodel, fpgamodule

onnx_model = make_multilayer(1, [8,8])[0]
#print(onnx_model)
spec = fpgamodule.FPGASpec(120, 600_000, 2_700_000, 100_000)
fpga_module = parsemodel.parse_model(onnx_model, 8, 8, spec)
#pprint(list(mod for mod in fpga_module.modules))
fpga_module.alloc_regs()
fpga_module.alloc_bram()
sv = fpga_module.make_sv()
with open("dummy_model.sv", "w") as f:
    f.write(sv)

In [5]:
from onnxruntime import InferenceSession
import onnxruntime as ort
onnx.save(onnx_model, "testmodel.onnx")
sess = InferenceSession("testmodel.onnx")
x_test = np.zeros(8).astype(np.int32)
x_test = ort.OrtValue.ortvalue_from_numpy(x_test)
res = sess.run(["Y_0"], {"X_0": x_test})
print(res)

[array([9, 0, 0, 0, 0, 2, 0, 6], dtype=int32)]


No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.2'
